In [31]:
import pandas as pd
from flaml import AutoML
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [32]:
df = pd.read_csv("C:\\Users\\LENOVO\\Downloads\\Compressed\\bank_marketing\\data\\bank-additional-full.csv", na_values='unknown')
df.head()

,age,job,marital,education,default,housing,loan,is_telephone_contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,y
0,56,housemaid,married,basic.4y,no,no,no,yes,may,mon,261,1,999,0,nonexistent,no
1,57,services,married,high.school,NaN,no,no,yes,may,mon,149,1,999,0,nonexistent,no
2,37,services,married,high.school,no,yes,no,yes,may,mon,226,1,999,0,nonexistent,no
3,40,admin.,married,basic.6y,no,no,no,yes,may,mon,151,1,999,0,nonexistent,no
4,56,services,married,high.school,no,no,yes,yes,may,mon,307,1,999,0,nonexistent,no


In [33]:
categorical_cols = ['day_of_week', 'month', 'education', 'job', 'default', 'is_telephone_contact', 'marital', 'housing', 'loan', 'poutcome']
df[categorical_cols] = df[categorical_cols].apply(lambda x: x.fillna(x.mode()[0]))
df

,age,job,marital,education,default,housing,loan,is_telephone_contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,y
0,56,housemaid,married,basic.4y,no,no,no,yes,may,mon,261,1,999,0,nonexistent,no
1,57,services,married,high.school,no,no,no,yes,may,mon,149,1,999,0,nonexistent,no
2,37,services,married,high.school,no,yes,no,yes,may,mon,226,1,999,0,nonexistent,no
3,40,admin.,married,basic.6y,no,no,no,yes,may,mon,151,1,999,0,nonexistent,no
4,56,services,married,high.school,no,no,yes,yes,may,mon,307,1,999,0,nonexistent,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,no,nov,fri,334,1,999,0,nonexistent,yes
41184,46,blue-collar,married,professional.course,no,no,no,no,nov,fri,383,1,999,0,nonexistent,no
41185,56,retired,married,university.degree,no,yes,no,no,nov,fri,189,2,999,0,nonexistent,no
41186,44,technician,married,professional.course,no,no,no,no,nov,fri,442,1,999,0,nonexistent,yes


In [34]:
continuous_cols = ['duration', 'age','pdays','campaign','previous']
binary_cols = ['default', 'is_telephone_contact','loan','housing']
nominal_cols = [ 'marital','job']
ordinal_cols = ['education','month','day_of_week','poutcome']

In [35]:
binary_df = df[binary_cols]
for col in binary_df:
    mapping ={'no': 0, 'yes':1, 0:0, 1:1}
    binary_df.loc[:, col] = binary_df[col].map(mapping)

binary_df.head()

,default,is_telephone_contact,loan,housing
0,0,1,0,0
1,0,1,0,0
2,0,1,0,1
3,0,1,0,0
4,0,1,1,0


In [36]:
for col in df[nominal_cols]:
    dummies = pd.get_dummies(df[nominal_cols],dtype=int, prefix=col+'_')

nominal_df = pd.get_dummies(df[nominal_cols],dtype=int,)
nominal_df.head()

,marital_divorced,marital_married,marital_single,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed
0,0,1,0,0,0,0,1,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,1,0,0,0
2,0,1,0,0,0,0,0,0,0,0,1,0,0,0
3,0,1,0,1,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,1,0,0,0


In [37]:
from sklearn.preprocessing import LabelEncoder
ordinal_df = df[ordinal_cols]
label_encoder = LabelEncoder()

for col in ordinal_df:
    if col == 'poutcome':
        mapping = {'failure': -1, 'nonexistent': 0, 'success': 1}
        ordinal_df[col] = ordinal_df[col].map(mapping)
    elif col == 'day_of_week':
        mapping = {'mon': 0, 'tue': 1, 'wed': 2, 'thu': 3, 'fri': 4,'sat':5,'sun':6}
        ordinal_df[col] = ordinal_df[col].map(mapping)
    elif col == 'month':
        mapping = {'jan': 0, 'feb': 1, 'mar': 2, 'apr': 3, 'may': 4, 'jun': 5,
                   'jul': 6, 'aug': 7, 'sep': 8, 'oct': 9, 'nov': 10, 'dec': 11}
        ordinal_df[col] = ordinal_df[col].map(mapping)
    elif col == 'education':
        mapping = {'illiterate': 0, 'basic.4y': 1, 'basic.6y': 2, 'basic.9y': 3,
                   'high.school': 4, 'professional.course': 5, 'university.degree': 6}
        ordinal_df[col] = ordinal_df[col].map(mapping)

        
ordinal_df.head()



C:\Users\LENOVO\AppData\Local\Temp\ipykernel_23520\3520781558.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ordinal_df[col] = ordinal_df[col].map(mapping)
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_23520\3520781558.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ordinal_df[col] = ordinal_df[col].map(mapping)
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_23520\3520781558.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

,education,month,day_of_week,poutcome
0,1,4,0,0
1,4,4,0,0
2,4,4,0,0
3,2,4,0,0
4,4,4,0,0


In [38]:
df.head()

,age,job,marital,education,default,housing,loan,is_telephone_contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,y
0,56,housemaid,married,basic.4y,no,no,no,yes,may,mon,261,1,999,0,nonexistent,no
1,57,services,married,high.school,no,no,no,yes,may,mon,149,1,999,0,nonexistent,no
2,37,services,married,high.school,no,yes,no,yes,may,mon,226,1,999,0,nonexistent,no
3,40,admin.,married,basic.6y,no,no,no,yes,may,mon,151,1,999,0,nonexistent,no
4,56,services,married,high.school,no,no,yes,yes,may,mon,307,1,999,0,nonexistent,no


In [39]:
import zipfile
import joblib

joblib.dump(categorical_cols, "categorical_cols")
joblib.dump(continuous_cols, "continuous_cols")
joblib.dump(binary_cols, "binary_cols")
joblib.dump(nominal_cols, "nominal_cols")
joblib.dump(ordinal_cols, "ordinal_cols")

binary_df.to_csv('binary_df.csv', index=False)
nominal_df.to_csv('nominal_df.csv', index=False)
ordinal_df.to_csv('ordinal_df.csv', index=False)

df.to_csv('df.csv', index=False)

def compress(file_names):
    print("File Paths:")
    print(file_names)
    compression = zipfile.ZIP_DEFLATED
    with zipfile.ZipFile("result.zip", mode="w") as zf:
        for file_name in file_names:
            zf.write('./' + file_name, file_name, compress_type=compression)

file_names = ["categorical_cols", "continuous_cols", "binary_cols", "nominal_cols", "ordinal_cols", \
              "binary_df.csv", "nominal_df.csv", "ordinal_df.csv", "df.csv", "bank_marketing.ipynb"]
compress(file_names)

File Paths:
['categorical_cols', 'continuous_cols', 'binary_cols', 'nominal_cols', 'ordinal_cols', 'binary_df.csv', 'nominal_df.csv', 'ordinal_df.csv', 'df.csv', 'bank_marketing.ipynb']
